In [1]:
from __future__ import print_function
from __future__ import division
import json
import py_crepe
import datetime
import numpy as np
import data_helpers
import data
import string
import pandas as pd
np.random.seed(0123)  # for reproducibility


Using Theano backend.


In [2]:
# set parameters:

subset = None

#Whether to save model parameters
save = False
model_name_path = 'params/crepe_model.json'
model_weights_path = 'params/crepe_model_weights.h5'

#Maximum length. Longer gets chopped. Shorter gets padded.
maxlen = 1014

#Model params
#Filters for conv layers
nb_filter = 128 #initially 256
#Number of units in the dense layer
dense_outputs = 512 #Initially 1024
#Conv layer kernel size
filter_kernels = [7, 7, 3, 3, 3, 3]
#Number of units in the final output layer. Number of classes.

#Compile/fit params
batch_size = 32
nb_epoch = 15

In [3]:
print('Loading data...')
#Expect x to be a list of sentences. Y to be a one-hot encoding of the
#categories.

### 515-1122-122 and 1573 with remove 6 layers
#authorlist=[121, 479 , 649 ]
#doc_id = 1470

authorlist=[ 104, 1430 , 1610]
doc_id = 3845
cat_output = len(authorlist) #binary in the last layer

# def main(authorlist, doc_id):
    
    
((trainX, trainY), (valX, valY)) = data_helpers.load_ag_data(authors = authorlist, docID = doc_id)

print('Creating vocab...')
vocab, reverse_vocab, vocab_size, check = data_helpers.create_vocab_set()


#trainX = data_helpers.encode_data(trainX, maxlen, vocab, vocab_size, check)
#test_data = data_helpers.encode_data(valX, maxlen, vocab, vocab_size, check)

print('Build model...')

classes = len(authorlist)

model = py_crepe.model(classes, filter_kernels, dense_outputs, maxlen, vocab_size, nb_filter)

Loading data...
Execution completed
Read completed
Number of rows: 33
author_id       int64
doc_content    object
dtype: object
Data Frame created: Shape: (4589, 2)
Author:   104  Size:  2212
Author:  1430  Size:  1143
Author:  1610  Size:  1234
Min: 1143
Max: 2212
Authors [104, 1430, 1610].
Found 3429 texts.
Found 3429 labels.
Creating vocab...
Build model...


In [4]:
vocab_size

1280

In [5]:
print('Fit model...')
initial = datetime.datetime.now()
for e in xrange(nb_epoch):
    xi, yi = data_helpers.shuffle_matrix(trainX, trainY)
    xi_test, yi_test = data_helpers.shuffle_matrix(valX, valY)
    if subset:
        batches = data_helpers.mini_batch_generator(xi[:subset], yi[:subset],
                                                    vocab, vocab_size, check,
                                                    maxlen,
                                                    batch_size=batch_size)
    else:
        batches = data_helpers.mini_batch_generator(xi, yi, vocab, vocab_size,
                                                    check, maxlen,
                                                    batch_size=batch_size)

    test_batches = data_helpers.mini_batch_generator(xi_test, yi_test, vocab,
                                                     vocab_size, check, maxlen,
                                                     batch_size=batch_size)

    accuracy = 0.0
    loss = 0.0
    step = 1
    start = datetime.datetime.now()
    print('Epoch: {}'.format(e))
    for x_train, y_train in batches:
        
        f = model.train_on_batch(x_train, y_train)
        loss += f[0]
        loss_avg = loss / step
        accuracy += f[1]
        accuracy_avg = accuracy / step
        if step % 100 == 0:
            print('  Step: {}'.format(step))
            print('\tLoss: {}. Accuracy: {}'.format(loss_avg, accuracy_avg))
        step += 1

    test_accuracy = 0.0
    test_loss = 0.0
    test_step = 1
    
    for x_test_batch, y_test_batch in test_batches:
        f_ev = model.test_on_batch(x_test_batch, y_test_batch)
        test_loss += f_ev[0]
        test_loss_avg = test_loss / test_step
        test_accuracy += f_ev[1]
        test_accuracy_avg = test_accuracy / test_step
        test_step += 1
    stop = datetime.datetime.now()
    e_elap = stop - start
    t_elap = stop - initial
    print('Epoch {}. Loss: {}. Accuracy: {}\nEpoch time: {}. Total time: {}\n'.format(e, test_loss_avg, test_accuracy_avg, e_elap, t_elap))

if save:
    print('Saving model params...')
    json_string = model.to_json()
    with open(model_name_path, 'w') as f:
        json.dump(json_string, f)

model.save_weights(model_weights_path)

Fit model...
Epoch: 0
Epoch 0. Loss: 0.699053371494. Accuracy: 0.750608766621
Epoch time: 0:03:57.297824. Total time: 0:03:57.381214

Epoch: 1
Epoch 1. Loss: 0.301732566208. Accuracy: 0.892045454545
Epoch time: 0:03:53.313655. Total time: 0:07:50.759249

Epoch: 2
Epoch 2. Loss: 0.168360232968. Accuracy: 0.938514609228
Epoch time: 0:03:52.478290. Total time: 0:11:43.302618

Epoch: 3
Epoch 3. Loss: 0.127565243366. Accuracy: 0.950284090909
Epoch time: 0:03:53.995484. Total time: 0:15:37.364066

Epoch: 4
Epoch 4. Loss: 0.0891448812009. Accuracy: 0.96875
Epoch time: 0:03:57.517184. Total time: 0:19:34.946098

Epoch: 5
Epoch 5. Loss: 0.103640308011. Accuracy: 0.963676948439
Epoch time: 0:03:53.844374. Total time: 0:23:28.854739

Epoch: 6
Epoch 6. Loss: 0.108765895902. Accuracy: 0.961241881956
Epoch time: 0:03:57.289922. Total time: 0:27:26.209448

Epoch: 7
Epoch 7. Loss: 0.0858706858271. Accuracy: 0.967329545455
Epoch time: 0:03:56.769772. Total time: 0:31:23.044480

Epoch: 8
Epoch 8. Loss: 